# **<center>Vehicle Detection with Background Subtraction and Contours.<center>**

    
In this tutorial we'll learn how to leverage Background Subtraction and Contours in order to detect moving objects.

### **Import the Libraries**
Let's First start by importing the libraries.

In [13]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from time import sleep
%matplotlib inline

## **Car Detection using Background Subtraction**

Background subtraction is a simple yet effective technique to extract objects from an image/video. Consider a highway on which cars are moving, and you want to extract each car. One easy way can be that you take a picture of the highway with the cars (called foreground image) then you also have an image saved in which the highway does not contain any cars (background image) and then you subtract the background image from the foreground to get the segmented mask of the cars and then use that mask to extract the cars.

<br>

<center>
<img src="https://drive.google.com/uc?export=download&id=1HDV7pH9iO2ChV2jwoAaKKOPJYZC7bJDB">
</center>


But in many cases you don't have a clear background image, an example of this can be a highway that is always busy, or maybe a walking destination that is always crowded. So in those cases, you can subtract the background by other means, for example, in the case of a video you can detect the movement of the object, so the objects which move can be foreground and the other part that remain static can be the background.

Several algorithms have been invented for this purpose. OpenCV has implemented a few such algorithms which are very easy to use. Let's see one of them.

### **BackgroundSubtractorMOG2**

It is a Background/Foreground Segmentation Algorithm, based on two papers by **Z.Zivkovic**, "[**Improved adaptive Gaussian mixture model for background subtraction**](https://ieeexplore.ieee.org/abstract/document/1333992)" (IEEE 2004) and "[**Efficient Adaptive Density Estimation per Image Pixel for the Task of Background Subtraction**](https://www.sciencedirect.com/science/article/abs/pii/S0167865505003521)" (Elsevier BV 2006). One important feature of this algorithm is that it provides better adaptability to varying scenes due to illumination changes etc.


**Function Syntax:**


> [**`object = cv2.createBackgroundSubtractorMOG2(history, varThreshold, detectShadows)`**](https://docs.opencv.org/4.2.0/de/de1/group__video__motion.html#ga2beb2dee7a073809ccec60f145b6b29c)

**Parameters:**

* **`history`** (optional) - It is the length of the history. Its default value is 500.


* **`varThreshold`** (optional) - It is the threshold on the squared distance between the pixel and the model to decide whether a pixel is well described by the background model. It does not affect the background update and its default value is 16.


* **`detectShadows`** (optional) - It is a boolean that determines whether the algorithm will detect and mark shadows or not. It marks shadows in gray color. Its default value is True. It decreases the speed a bit, so if you do not need this feature, set the parameter to false. 

**Returns:**

* **`object`** - It is the MOG2 Background Subtractor.

In [3]:
# load a video
cap = cv2.VideoCapture('media/videos/vtest.avi')

# you can optionally work on the live web cam
cap = cv2.VideoCapture(0)

# create the background object, you can choose to detect shadows or not (if True they will be shown as gray)
backgroundobject = cv2.createBackgroundSubtractorMOG2( history = 2, detectShadows = True )

while(1):
    ret, frame = cap.read()  
    if not ret:
        break
        
    # apply the background object on each frame
    fgmask = backgroundobject.apply(frame)

    # also extracting the real detected foreground part of the image (optional)
    real_part = cv2.bitwise_and(frame,frame,mask=fgmask)
    
    # making fgmask 3 channeled so it can be stacked with others
    fgmask_3 = cv2.cvtColor(fgmask, cv2.COLOR_GRAY2BGR)
    
    # Stack all three frames and show the image
    stacked = np.hstack((fgmask_3,frame,real_part))
    cv2.imshow('All three',cv2.resize(stacked,None,fx=0.65,fy=0.65))
 
    k = cv2.waitKey(30) &  0xff
    if k == 27:
        break
   
cap.release()
cv2.destroyAllWindows()

[ WARN:0] global ../modules/videoio/src/cap_gstreamer.cpp (935) open OpenCV | GStreamer warning: Cannot query video position: status=0, value=-1, duration=-1


## Creating the Vehicle Detection Application.

To Perform the complete background Subtraction based contour detection, we'll be performing these simple steps


**1)** First, we will load a video using the function [**`cv2.VideoCapture()`**](https://docs.opencv.org/3.4/d8/dfe/classcv_1_1VideoCapture.html#a949d90b766ba42a6a93fe23a67785951) and create a background subtractor object using the function **`cv2.createBackgroundSubtractorMOG2()`**.


**2)** Then we will use the [**`backgroundsubtractor.apply()`**](https://docs.opencv.org/3.4/d7/d7b/classcv_1_1BackgroundSubtractorMOG2.html#a682adde901148d85450435e6cc0de4a1) method to get the segmented masks for the frames of the video after reading the frames one by one using the function [**`cv2.VideoCapture.read()`**](https://docs.opencv.org/3.4/d8/dfe/classcv_1_1VideoCapture.html#a473055e77dd7faa4d26d686226b292c1). 

**3)** We will then apply thresholding on the mask using the function [**`cv2.threshold()`**](https://docs.opencv.org/3.4/d7/d1b/group__imgproc__misc.html#gae8a4a146d1ca78c626a53577199e9c57)  to get rid of shadows and then perform Erosion and Dilation to improve the mask further using the functions [**`cv2.erode()`**](https://docs.opencv.org/3.4/d4/d86/group__imgproc__filter.html#gaeb1e0c1033e3f6b891a25d0511362aeb) and [**`cv2.dilate()`**](https://docs.opencv.org/3.4/d4/d86/group__imgproc__filter.html#ga4ff0f3318642c4f469d0e11f242f3b6c).

**4)** Then we will use the function [**`cv2.findContours()`**](https://docs.opencv.org/3.4/d3/dc0/group__imgproc__shape.html#ga17ed9f5d79ae97bd4c7cf18403e1689a) to detect the contours on the mask image and convert the contour coordinates into bounding box coordinates for each car in the frame using the function [**`cv2.boundingRect()`**](https://docs.opencv.org/3.4/d3/dc0/group__imgproc__shape.html#ga103fcbda2f540f3ef1c042d6a9b35ac7). We will check if the area of the contour is greater than a threshold to make sure that it's a car which we will find using the function [**`cv2.contourArea()`**](https://docs.opencv.org/3.4/d3/dc0/group__imgproc__shape.html#ga2c759ed9f497d4a618048a2f56dc97f1).

**5)** After that we will use the functions [**`cv2.rectangle()`**](https://docs.opencv.org/3.4/d6/d6e/group__imgproc__draw.html#ga07d2f74cadcf8e305e810ce8eed13bc9) and [**`cv2.putText()`**](https://docs.opencv.org/3.4/d6/d6e/group__imgproc__draw.html#ga5126f47f883d730f633d74f07456c576) to draw and label the bounding boxes on each frame and then we will extract the foreground part of the video with the help of the segmented mask using the function [**`cv2.bitwise_and()`**](https://docs.opencv.org/3.4/d2/de8/group__core__array.html#ga60b4d04b251ba5eb1392c34425497e14).

In [19]:
# load a video
video = cv2.VideoCapture('trimmed1.mp4')

# You can set custom kernel size if you want.
kernel = None

# Initialize the background object.
backgroundObject = cv2.createBackgroundSubtractorMOG2(varThreshold=10,detectShadows = True)

while True:
    
    # Read a new frame.
    ret, frame = video.read()

    # Check if frame is not read correctly.
    if not ret:
        break

    # Apply the background object on the frame to get the segmented mask. 
    fgmask = backgroundObject.apply(frame)
    #initialMask = fgmask.copy()
    
    # Perform thresholding to get rid of the shadows.
    _, fgmask = cv2.threshold(fgmask, 250, 255, cv2.THRESH_BINARY)
    #noisymask = fgmask.copy()
    
    # Apply some morphological operations to make sure you have a good mask
    fgmask = cv2.erode(fgmask, kernel, iterations = 1)
    fgmask = cv2.dilate(fgmask, kernel, iterations = 2)
    
    # Detect contours in the frame.
    contours, _ = cv2.findContours(fgmask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Create a copy of the frame to draw bounding boxes around the detected cars.
    frameCopy = frame.copy()
    sleep(0.05)
    # loop over each contour found in the frame.
    for cnt in contours:
        
        # Make sure the contour area is somewhat higher than some threshold to make sure its a car and not some noise.
        if cv2.contourArea(cnt) > 400:
            
            # Retrieve the bounding box coordinates from the contour.
            x, y, width, height = cv2.boundingRect(cnt)
            
            # Draw a bounding box around the car.
            cv2.rectangle(frameCopy, (x , y), (x + width, y + height),(0, 0, 255), 2)
            
            # Write Car Detected near the bounding box drawn.
            cv2.putText(frameCopy, 'Car Detected', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0,255,0), 1, cv2.LINE_AA)
    
    # Extract the foreground from the frame using the segmented mask.
    foregroundPart = cv2.bitwise_and(frame, frame, mask=fgmask)
        
    # Stack the original frame, extracted foreground, and annotated frame. 
    stacked = np.hstack((frame, foregroundPart, frameCopy))

    # Display the stacked image with an appropriate title.
    cv2.imshow('Original Frame, Extracted Foreground and Detected Cars', cv2.resize(stacked, None, fx=0.5, fy=0.5))
    #cv2.imshow('initial Mask', initialMask)
    #cv2.imshow('Noisy Mask', noisymask)
    #cv2.imshow('Clean Mask', fgmask)


    # Wait until a key is pressed.
    # Retreive the ASCII code of the key pressed
    k = cv2.waitKey(1) & 0xff
    
    # Check if 'q' key is pressed.
    if k == ord('q'):
        
        # Break the loop.
        break

# Release the VideoCapture Object.
video.release()

# Close the windows.q
cv2.destroyAllWindows()

There are many other background subtraction algorithm in OpenCV that you can check out [here](https://docs.opencv.org/4.1.0/d2/d55/group__bgsegm.html) and [here](https://docs.opencv.org/4.1.0/de/de1/group__video__motion.html)  along with their parameters and other details as well.